# Overview:

# We have already detected the faces using haarscades  (done)

    -1.Store the face information into numpy array.

    -2. Flatten the largest face image and save in numpy array.

    -3. Repeate the above for multiple people to generate training data and
    store them all in the "data" file which having extension ".npy".



In [5]:
import cv2
import numpy as np
cap=cv2.VideoCapture(0)

#face detection
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    print(faces)
    
    for face in faces:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    
    cv2.imshow("Frame",frame)
    
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()


# In the output we are geting the Frame dimension ,along with the online vedion stream
    

[[252 160 163 163]]
()
()
()
()
()
()
()
()
[[243 158 171 171]]
()
()
[[244 148 179 179]]
[[244 148 179 179]]
[[249 154 170 170]]
()
()
()
()
()
()
()
()
()
()
[[261 140 191 191]]
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[253 148 179 179]]
[[253 148 179 179]]
()
()
()
()
()
()
()
()
()
[[228 147 188 188]]
[[230 149 177 177]]
[[221 145 188 188]]
()
()
()
()
()
()
[[230 138 196 196]]
()
()
()
()
[[251 140 201 201]]
()
[[267 142 191 191]]
()
()
()
()
()
[[267 141 200 200]]
[[263 144 191 191]]
()
()
[[266 141 196 196]]
[[266 148 185 185]]
[[266 146 188 188]]
[[269 147 185 185]]
[[269 147 185 185]]
[[265 146 190 190]]
[[266 143 194 194]]
[[266 143 194 194]]
()
[[276 151 179 179]]
[[276 151 179 179]]
()
()
()
[[266 144 191 191]]
[[268 146 188 188]]
[[268 146 185 185]]
[[262 140 196 196]]
[[265 146 188 188]]
()
()
()
[[260 138 198 198]]
[[259 136 201 201]]
[[252 134 205 205]]
[[257 137 198 198]]
[[257 137 198 198]]
[[256 129 212 212]]
[[264 13

In [6]:

import numpy as np
cap=cv2.VideoCapture(0)

#face detection
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path='./data/'
file_name=input("Enter the name of the person : ")

while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
    
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    
    #pick the last face( because it is the larges face acc to area(f(2) * f(3))
    for face in faces[-1:]:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        #extract ( crop the required face): region of interst
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        skip+=1
        if skip%10==0:
            face_data.append(face_section)
            print(len(face_data))
        
        
    cv2.imshow("Frame",frame)
    #cv2.imshow("Face section",face_section)
    #stor every 10 face
   
   # cv2.imshow("face section",face_section)

    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
        
        
#convert our face list arrray into nupy array
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)


# save this data into file system

np.save(dataset_path+file_name+'.npy',face_data)
print("Data successfully save at"+dataset_path+file_name+'.npy')

cap.release()
cv2.destroyAllWindows()



Enter the name of the person : Firoz THakur
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
(47, 30000)
Data successfully save at./data/Firoz THakur.npy


# The previous says that 17 frame of the face have been saved in the file Firoz THakur.npy

# Now, Testing Phase : -

# 1. Recognise Faces using some classification algorithm - like logistic ,KNN,SVM etc .

# 2. Loaded the training  data ( numpy arrays of all the person )
    - x - values are stored in the numpy arrays.
    -y  - y-person we need stored in the  numpuy arrays.

# 3. Use the KNN to find the prediction of face.

# 4. Map the predicted ID to Name of user

# 5. Display the prediction on the screen - bounding box and name.

# Importing the libraries

In [3]:
import cv2
import numpy as np
import os


# KNN Algorithm to find the prediction of Face :

In [7]:
def distance(v1,v2):
    return np.sqrt(((v1-v2)**2).sum())


def knn(train,test,k=5):
    dist=[]
    
    for i in range(train.shape[0]):
        # get the vector and label
        ix=train[i,:-1]
        iy=train[i,-1]
        # compute the distance from the point
        d=distance(test,ix)
        dist.append([d,iy])
        #sort based on distance and get top k
    
    dk=sorted(dist,key=lambda x:x[0])[:k]
    
    #retrieve only the labels
    
    labels=np.array(dk)[:,-1]
    
    #gett freq of each labels
    
    output=np.unique(labels,return_counts=True)
    
    #find max frq and corresponding label
    index=np.argmax(output[1])
    return output[0][index]

# Testing phase

In [8]:
import cv2
import numpy as np
import os

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path='./data/'   
labels=[]
class_id=0
names={}  # dic
    
#data preparation
for fx in os.listdir(dataset_path):
    
    if fx.endswith('.npy'):
        
        #create a mapping btw classid and name
        
        names[class_id]=fx[:-4]
        print("Loaded "+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #create labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)

face_dataset=np.concatenate(face_data,axis=0)

face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
print(face_dataset.shape)
print(face_labels.shape)


trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)
        
        
        
        
        
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path='./data/'    
labels=[]
class_id=0
names={}
    
#data preparation

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #create a mapping btw classid and name
        names[class_id]=fx[:-4]
        print("Loaded "+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #create labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)

face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
print(face_dataset.shape)
print(face_labels.shape)


trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)
        
        
        
        
        
        
        
# testing : and calling to KNN         
        
while True:        
        ret,frame=cap.read()
        if ret==False:
            continue
        
        faces=face_cascade.detectMultiScale(frame,1.3,5)


        for face in faces:
            x,y,w,h=face
        
            #get the face ROI
            offset=10
            face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
            face_section=cv2.resize(face_section,(100,100))
        
            #predicted label(out)
            out=knn(trainset,face_section.flatten())
        
        #display on the screen the name rectangle around it
        
            pred_name=names[int(out)]
            cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
    
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    
        cv2.imshow("Face",frame)
    
        key=cv2.waitKey(1) & 0xff
        if key==ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()


Loaded abhi chotu.npy
Loaded BoHemia sir.npy
Loaded Chaman Lal.npy
Loaded chinalo.npy
Loaded diksha rana.npy
Loaded FIROZ THakur.npy
Loaded Firoz.npy
Loaded golu.npy
Loaded Kalpna Beta.npy
Loaded Kamal Mehta.npy
Loaded Kotta chinn.npy
Loaded Monika Thakur.npy
Loaded parekshit NiGa.npy
Loaded Pritam vardhan.npy
Loaded Rakesh Thakur.npy
Loaded shanu.npy
Loaded shubham bhai.npy
Loaded sonu singh.npy
(527, 30000)
(527, 1)
(527, 30001)
Loaded abhi chotu.npy
Loaded BoHemia sir.npy
Loaded Chaman Lal.npy
Loaded chinalo.npy
Loaded diksha rana.npy
Loaded FIROZ THakur.npy
Loaded Firoz.npy
Loaded golu.npy
Loaded Kalpna Beta.npy
Loaded Kamal Mehta.npy
Loaded Kotta chinn.npy
Loaded Monika Thakur.npy
Loaded parekshit NiGa.npy
Loaded Pritam vardhan.npy
Loaded Rakesh Thakur.npy
Loaded shanu.npy
Loaded shubham bhai.npy
Loaded sonu singh.npy
(527, 30000)
(527, 1)
(527, 30001)


![](baba11.jpg)

# Test the system for different input faces.

[click here to watch the vedio of output result](https://drive.google.com/file/d/151HoxxH8-s2d7s0_g2uaw_-LQuBbEsvg/view)